In [1]:
import boto3
from configparser import ConfigParser
import json
import pandas as pd
import io

In [2]:
# Carregar as configurações do arquivo ini
config = ConfigParser()
config.read("../config/config.ini")

['../config/config.ini']

In [3]:
# Configurações para conexão com o MinIO/S3
endpoint = config.get("MinIO", "endpoint")
access_key = config.get("MinIO", "access_key")
secret_key = config.get("MinIO", "secret_key")
bucket_trust = config.get("Bucket", "bucket_trust")
bucket_context = config.get("Bucket", "bucket_context")

# Nome do arquivo a ser lido
source_filename = config.get("FILE", "censo_municipios_csv")
target_filename = config.get("FILE", "censo_saude_csv")

In [4]:
print("endpoint:", endpoint)
print("access_key:", access_key)
print("secret_key:", secret_key)
print("bucket_trust:", bucket_trust)
print("bucket_context:", bucket_context)
print("source_filename:", source_filename)
print("target_filename:", target_filename)

endpoint: http://minio:9000
access_key: aulafia
secret_key: aulafia@123
bucket_trust: trust
bucket_context: context
source_filename: CensoMunicipios.csv
target_filename: CensoSaude.csv


In [5]:
# Inicializar o cliente boto3 para S3
minio_client = boto3.client("s3", 
                            endpoint_url=endpoint,
                            aws_access_key_id=access_key,
                            aws_secret_access_key=secret_key
)

In [7]:
# Baixar o arquivo CSV do bucket 'context'
response = minio_client.get_object(Bucket=bucket_context,
                         Key=source_filename)
csv_content = response['Body'].read()

In [8]:
# Carregar o conteúdo CSV em um DataFrame do pandas
data_frame = pd.read_csv(io.BytesIO(csv_content))

In [9]:
# Selecionar apenas as 13 primeiras colunas
selected_columns = data_frame.iloc[:, :13]

In [10]:
# Criar um fluxo de gravação em buffer para o novo arquivo CSV
output_buffer = io.StringIO()

In [11]:
# Salvar o DataFrame reduzido como um arquivo CSV no buffer
selected_columns.to_csv(output_buffer, index=False, encoding='utf-8')

In [12]:
# Obter o conteúdo do buffer como bytes codificados em UTF-8
encoded_csv_content = output_buffer.getvalue().encode('utf-8')

In [13]:
# Enviar o novo arquivo CSV codificado para o bucket de destino
minio_client.put_object(Bucket=bucket_trust, Key=target_filename, Body=encoded_csv_content)

{'ResponseMetadata': {'RequestId': '177F63F77B2DF509',
  'HostId': 'dd9025bab4ad464b049177c95eb6ebf374d3b3fd1af9251148b658df7ac2e3e8',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'accept-ranges': 'bytes',
   'content-length': '0',
   'etag': '"0b7aae5236703f45efd4a251fa5b8d14"',
   'server': 'MinIO',
   'strict-transport-security': 'max-age=31536000; includeSubDomains',
   'vary': 'Origin, Accept-Encoding',
   'x-amz-id-2': 'dd9025bab4ad464b049177c95eb6ebf374d3b3fd1af9251148b658df7ac2e3e8',
   'x-amz-request-id': '177F63F77B2DF509',
   'x-content-type-options': 'nosniff',
   'x-xss-protection': '1; mode=block',
   'date': 'Mon, 28 Aug 2023 00:18:19 GMT'},
  'RetryAttempts': 0},
 'ETag': '"0b7aae5236703f45efd4a251fa5b8d14"'}